### 머신러닝 첫번째 - 데이터 정규화

In [1]:
import oracledb

# Oracle Instant Client 라이브러리 경로
client_dir = r"C:\\Users\\user\\Desktop\\KSM\\Tools\\instantclient-basic-windows.x64-19.25.0.0.0dbru\\instantclient_19_25"
oracledb.init_oracle_client(lib_dir=client_dir)

# 접속 정보 입력
username = "libra"
password = "ksm0923"
dsn = "localhost:1521/XE"

# 접속 시도
try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = conn.cursor()
    print("🔗 Oracle DB 접속 성공")
except Exception as e:
    print("❌ Oracle DB 연결 실패:", e)

🔗 Oracle DB 접속 성공


In [17]:
import pandas as pd

schema = "LIBRA"  # Oracle 사용자 스키마명 (대문자)
years = list(range(2014, 2025))  # 2014 ~ 2024
prefixes = [f"NUM0{i}_" for i in range(1, 6)]  # NUM01_ ~ NUM05_

result = []

for idx, year in enumerate(years):
    prefix = prefixes[idx % len(prefixes)]  # 반복적으로 테이블 번호 선택
    table_name = f"{prefix}{year}"

    sc_query = f"""
        SELECT COUNT(*) FROM ALL_TAB_COLUMNS
        WHERE OWNER = '{schema}' AND TABLE_NAME = '{table_name.upper()}'
        AND REGEXP_LIKE(COLUMN_NAME, '(^|_)SC$', 'i')
        AND NOT REGEXP_LIKE(COLUMN_NAME, 'SCC$|SCP$', 'i')
    """

    scc_query = f"""
        SELECT COUNT(*) FROM ALL_TAB_COLUMNS
        WHERE OWNER = '{schema}' AND TABLE_NAME = '{table_name.upper()}'
        AND REGEXP_LIKE(COLUMN_NAME, 'SCC$', 'i')
    """

    scp_query = f"""
        SELECT COUNT(*) FROM ALL_TAB_COLUMNS
        WHERE OWNER = '{schema}' AND TABLE_NAME = '{table_name.upper()}'
        AND REGEXP_LIKE(COLUMN_NAME, 'SCP$', 'i')
    """

    try:
        cursor.execute(sc_query)
        sc_count = cursor.fetchone()[0]
        cursor.execute(scc_query)
        scc_count = cursor.fetchone()[0]
        cursor.execute(scp_query)
        scp_count = cursor.fetchone()[0]

        result.append({
            "연도": year,
            "테이블명": table_name,
            "SC 컬럼 수": sc_count,
            "SCC 컬럼 수": scc_count,
            "SCP 컬럼 수": scp_count
        })
    except Exception as e:
        result.append({
            "연도": year,
            "테이블명": table_name,
            "SC 컬럼 수": f"실패: {str(e)}",
            "SCC 컬럼 수": "",
            "SCP 컬럼 수": ""
        })

# 결과 출력
df_result = pd.DataFrame(result)
display(df_result)

,연도,테이블명,SC 컬럼 수,SCC 컬럼 수,SCP 컬럼 수
0,2014,NUM01_2014,0,1,0
1,2015,NUM02_2015,0,1,0
2,2016,NUM03_2016,0,1,0
3,2017,NUM04_2017,0,0,3
4,2018,NUM05_2018,0,0,2
5,2019,NUM01_2019,0,0,1
6,2020,NUM02_2020,0,1,0
7,2021,NUM03_2021,0,0,1
8,2022,NUM04_2022,2,0,2
9,2023,NUM05_2023,0,2,0


[]


SyntaxError: invalid syntax (270706406.py, line 1)